In [14]:
from ast import literal_eval
import numpy as np
import os
import pandas as pd
import pickle
from scipy.spatial import cKDTree
import sys

from rex.utilities import to_records_array

cwd = os.getcwd()


def check_value(dict_in):
    dict_out = {}
    for key, value in dict_in.items():
        if isinstance(value, dict):
            for k, v in value.items():
                dict_out[key] = check_value(value)
        else:
            if isinstance(value, str):
                if '/' in value:
                    pass
                else:
                    dict_out[key] = value
            else:
                dict_out[key] = value
    return dict_out


def remove_paths(dict_in):
    dict_out = {}
    for key, value in dict_in.items():
        if isinstance(value, dict):
            for k, v in value.items():
                dict_out[key] = check_value(value)
        else:
            if isinstance(value, str):
                if '/' in value:
                    pass
                else:
                    dict_out[key] = value
            else:
                dict_out[key] = value

    return dict_out


def get_size(item):
    return sys.getsizeof(item.copy()) * 10**-6


def get_chunk_size(item, chunk_size=2.0, thresh=0.1):
    item_size = get_size(item)
    item_len = len(item)
    chunk_start = int(item_len // (item_size / chunk_size))
    i = 0
    while True:
        if item_len % (chunk_start + i) == 0:
            chunk = chunk_start + i
            break
        elif item_len % (chunk_start - i) == 0:
            chunk = chunk_start - i
            break
        else:
            offset_size = get_size(item[:i].copy())
            if offset_size > chunk_size * thresh:
                chunk = chunk_start
                print('No unique chunk found')
                break
            else:
                i += 1
    
    chunk_size = get_size(item[:chunk].copy())
    print('Chunk size ({}, ) = {:.2f} MB'.format(chunk, chunk_size))
    return chunk


def round_to(num, round_val):
    round_num = num - num % round_val
    return int(round_num)


def calc_chunks(t_chunk, dtype, chunk_size=2*10**6, round_to=None):
    pixel_size = np.dtype(dtype).itemsize
    s_chunk = chunk_size / (t_chunk * pixel_size)
    
    if round_to:
        s_chunk = round_to(s_chunk, round_to)
    else:
        s_chunk = int(np.floor(s_chunk))
    
    return (t_chunk, s_chunk)


box_dir = '/Users/mrossol/Box/HSDS/wave/West_Coast'

# West Coast
## Meta Data

In [3]:
meta_path = os.path.join(box_dir, 'west_coast_meta.csv')
columns = {'dist': 'distance_to_shore', 'depth': 'water_depth'}
meta = pd.read_csv(meta_path).rename(columns=columns)
meta

FileNotFoundError: [Errno 2] File /Users/mrossol/Box/HSDS/wave/west_coast_meta.csv does not exist: '/Users/mrossol/Box/HSDS/wave/west_coast_meta.csv'

In [8]:
meta_path = os.path.join(box_dir, 'west_coast_meta.csv')
columns = {'dist': 'distance_to_shore', 'depth': 'water_depth'}
meta = pd.read_csv(meta_path).rename(columns=columns)
meta = meta.set_index('gid')

meta_arr = CreateHSDS.to_records_array(meta)

out_path = os.path.join(box_dir, 'west_coast_meta.npy')
np.save(out_path, meta_arr)
meta_arr

rec.array([(  6.7184, 48.8641, -125.386,  993.09814 , -8, b'Canada'),
           ( 28.6991, 48.8418, -125.446, 5096.5234  , -8, b'Canada'),
           ( 49.4571, 48.8193, -125.504, 6850.4214  , -8, b'Canada'), ...,
           (  0.1   , 37.4631, -122.026,   35.939137, -8, b'California'),
           (-99.    , 37.4615, -122.026,    0.      , -8, b'California'),
           (  0.1   , 37.4614, -122.025,    0.      , -8, b'California')],
          dtype=[('water_depth', '<f4'), ('latitude', '<f4'), ('longitude', '<f4'), ('distance_to_shore', '<f4'), ('timezone', '<i2'), ('jurisdiction', 'S20')])

## Time_Index and Meta Chunk Sizes

In [4]:
# Hourly
time_index = np.array(pd.date_range('2012-01-01 00:00:00', '2012-12-31 23:00:00', freq='3h').astype(str),
                      'S20')

print(time_index.shape)
t_size = get_size(time_index)
print('time_index size = {:.2f} MB'.format(t_size))

(2928,)
time_index size = 0.06 MB


In [5]:
meta_path = os.path.join(box_dir, 'west_coast_meta.npy')
meta_data = np.load(meta_path)

m_size = sys.getsizeof(meta_data) * 10**-6
print('meta size = {:.2f} MB'.format(m_size))

m_chunks = get_chunk_size(meta_data, thresh=0.1)

meta size = 26.60 MB
No unique chunk found
Chunk size (52631, ) = 2.00 MB


In [6]:
meta_path = os.path.join(box_dir, 'west_coast_meta.npy')
meta_data = np.load(meta_path, allow_pickle=True)
lat_lon = pd.DataFrame(meta_data)
lat_lon = lat_lon[['latitude', 'longitude']].values.copy()

c_size = sys.getsizeof(lat_lon) * 10**-6
print('lat_lon size = {:.2f} MB'.format(c_size))

c_chunks = get_chunk_size(lat_lon, thresh=0.1)

lat_lon size = 5.60 MB
No unique chunk found
Chunk size (249994, ) = 2.00 MB


## Variable Attributes

In [59]:
names = {'Depth': 'water_depth',
         'Dir': 'mean_wave_direction',
         'Hsig': 'significant_wave_height',
         'Period': 'mean_absolute_period',
         'RTpeak': 'peak_period',
         'Tm02': 'mean_zero-crossing_period',
         'Tm_10': 'energy_period',
         'd': 'directionality_coefficient',
         'djdmax': 'maximum_energy_direction',
         'owp': 'omni-directional_wave_power',
         'sw': 'spectral_width'}

descriptions = {'Depth': 'Grid node depth',
         'Dir': 'Direction Normal to the Wave Crests',
         'Hsig': 'Calculated as the zeroth spectral moment (i.e., H_m0)',
         'Period': 'Resolved Spectral Moment (m_0/m_1)',
         'RTpeak': 'The period associated with the maximum value of the wave energy spectrum',
         'Tm02': 'Total wave energy flux from all directions',
         'Tm_10': 'Spectral width characterizes the relative spreading of energy in the wave spectrum. Large values indicate a wider spectral peak',
         'd': 'Fraction of total wave energy travelling in the "direction of maximum wave power" direction',
         'djdmax': 'The direction from which the most wave energy is travelling',
         'owp': 'Total wave energy flux from all directions',
         'sw': 'Spectral width characterizes the relative spreading of energy in the wave spectrum. Large values indicate a wider spectral peak',}

units = {'Depth': 'm',
         'Dir': 'deg',
         'Hsig': 'm',
         'Period': 's',
         'RTpeak': 's',
         'Tm02': 's',
         'Tm_10': 's',
         'd': '',
         'djdmax': 'deg',
         'owp': 'W/m',
         'sw': '',}

src_names = {k : k for k in names}
src_names['directional_wave_spectrum'] = 'energy'

SWAN_names = {'Depth': 'DEPTH',
         'Dir': 'DIR',
         'Hsig': 'HSIGN',
         'Period': 'PER',
         'RTpeak': 'RTP',
         'Tm02': 'TMM10',
         'Tm_10': 'TM02',
         'd': '',
         'djdmax': '',
         'owp': '',
         'sw': '',}

IEC_names = {'Depth': 'h',
         'Dir': 'Sigma',
         'Hsig': 'H_s',
         'Period': 'T_p',
         'RTpeak': 'T_p',
         'Tm02': 'T_02',
         'Tm_10': 'T_e',
         'd': 'd',
         'djdmax': 'Jsigma_Jmax',
         'owp': 'J',
         'sw': 'epsilon_o',}

references = {'Depth': 'SWAN Manual',
         'Dir': 'SWAN Manual',
         'Hsig': 'SWAN Manual, IEC62600-101',
         'Period': 'SWAN Manual',
         'RTpeak': 'SWAN Manual',
         'Tm02': 'SWAN Manual, IEC62600-101',
         'Tm_10': 'IEC62600-101',
         'd': 'IEC62600-101',
         'djdmax': 'IEC62600-101',
         'owp': 'IEC62600-101',
         'sw': 'IEC62600-101',}

dimensions = {'Depth': ['position'],
         'Dir': ['time', 'position'],
         'Hsig': ['time', 'position'],
         'Period': ['time', 'position'],
         'RTpeak': ['time', 'position'],
         'Tm02': ['time', 'position'],
         'Tm_10': ['time', 'position'],
         'd': ['time', 'position'],
         'djdmax': ['time', 'position'],
         'owp': ['time', 'position'],
         'sw': ['time', 'position'], }

In [3]:
path = os.path.join(cwd, 'h5_attrs/west_coast_attrs.json')
wave_attrs = pd.read_json(path)
wave_attrs

,attrs,dtype,chunks
Depth,"{'chunksize': [48, 45000], 'dtype': 'float32',...",float32,"[48, 45000]"
Dir,"{'chunksize': [48, 45000], 'dtype': 'float32',...",float32,"[48, 45000]"
Hsig,"{'chunksize': [48, 45000], 'dtype': 'float32',...",float32,"[48, 45000]"
Period,"{'chunksize': [48, 45000], 'dtype': 'float32',...",float32,"[48, 45000]"
PkDir,"{'chunksize': [48, 45000], 'dtype': 'float32',...",float32,"[48, 45000]"
RTpeak,"{'chunksize': [48, 45000], 'dtype': 'float32',...",float32,"[48, 45000]"
Tm02,"{'chunksize': [48, 45000], 'dtype': 'float32',...",float32,"[48, 45000]"
Tm_10,"{'chunksize': [48, 45000], 'dtype': 'float32',...",float32,"[48, 45000]"
coordinates,"{'chunksize': 'Not Chunked', 'dtype': 'float32...",float32,None
d,"{'chunksize': [48, 45000], 'dtype': 'float32',...",float32,"[48, 45000]"


In [3]:
path = os.path.join(cwd, 'h5_attrs/west_coast_attrs.json')
wave_attrs = pd.read_json(path)

wave_attrs = wave_attrs.drop('PkDir')
wave_attrs['name'] = None

# 8 week hourly chunks
t_chunk = int(8 * 7 * 24 / 3)
for var, row in wave_attrs.iterrows():

    if var == 'time_index':
        wave_attrs.at[var, 'attrs'] = {'freq': '3h', 'timezone': 'UTC', 'units': 'GMT', 'dimensions': ['time']}
        wave_attrs.at[var, 'chunks'] = None
    elif var == 'coordinates':
        wave_attrs.at[var, 'chunks'] = (249994, 2)
        attrs  = {'description': '(latitude, longitude) using Datum: NAD83',
                  'src_name': '(Xp, Yp)', 'units': '(deg, deg)', 'dimensions': ['position']}
        wave_attrs.at[var, 'attrs'] = attrs
    else:
        if var == 'Depth':
            chunks = None
        else:
            chunks = calc_chunks(t_chunk, row['dtype'])

        wave_attrs.at[var, 'chunks'] = chunks
        wave_attrs.at[var , 'name'] = names[var]
        wave_attrs.at[var, 'attrs'] = {'description': descriptions[var],
                                       'dimensions': dimensions[var],
                                       'units': units[var],
                                       'src_name': src_names[var],
                                       'SWAN_name': SWAN_names[var],
                                       'IEC_name': IEC_names[var]}

# Meta data
wave_attrs.at['meta', 'chunks'] = (52631, )
wave_attrs.at['meta', 'dtype'] = None
wave_attrs.at['meta', 'name'] = None
wave_attrs.at['meta', 'attrs'] = {'dimensions': ['position']}

# Global attributes
wave_attrs.at['global', 'chunks'] = None
wave_attrs.at['global', 'dtype'] = None
wave_attrs.at['global', 'name'] = None
wave_attrs.at['global', 'attrs'] = {'ref_SWAN-Manual': "SWAN Team, SWAN: User Manual, Delft University of Technology, Delft, The Netherlands, Cycle III Version 41.31, 2019.",
"ref_IEC62600-101": "International Electrotechnical Commission, Marine energy - Wave, tidal and other water current converters - Part 101: Wave energy resource assessment and characterization, Technical Specification 62600–101, 2015.",
"ref_Wu-Wang-Yang-Garcia-Medina-2020": "W.C. Wu, T. Wang, Z. Yang, and G. García-Medina, “Development and validation of a high-resolution regional wave hindcast model for U.S. West Coast wave resource characterization,” Renewable Energy, vol. 152, pp. 736–753, Jun. 2020.",
"source": "PNNL2019", "version": "v1.0.0"}

path = os.path.join(cwd, 'hsds_attrs/west_coast_hsds_attrs.json')
wave_attrs.to_json(path, indent=4)
wave_attrs

,attrs,dtype,chunks,name
Depth,"{'description': 'Grid node depth', 'dimensions...",float32,None,water_depth
Dir,{'description': 'Direction Normal to the Wave ...,float32,"(448, 1116)",mean_wave_direction
Hsig,{'description': 'Calculated as the zeroth spec...,float32,"(448, 1116)",significant_wave_height
Period,{'description': 'Resolved Spectral Moment (m_0...,float32,"(448, 1116)",mean_absolute_period
RTpeak,{'description': 'The period associated with th...,float32,"(448, 1116)",peak_period
Tm02,{'description': 'Total wave energy flux from a...,float32,"(448, 1116)",mean_zero-crossing_period
Tm_10,{'description': 'Spectral width characterizes ...,float32,"(448, 1116)",energy_period
coordinates,"{'description': '(latitude, longitude) using D...",float32,"(249994, 2)",None
d,{'description': 'Fraction of total wave energy...,float32,"(448, 1116)",directionality_coefficient
djdmax,{'description': 'The direction from which the ...,float32,"(448, 1116)",maximum_energy_direction


# Virtual Buoy
## West Coast

In [15]:
meta_path = os.path.join(box_dir, 'west_coast_meta.csv')
columns = {'dist': 'distance_to_shore', 'depth': 'water_depth'}
meta = pd.read_csv(meta_path).rename(columns=columns)
meta = meta.set_index('gid')
tree = cKDTree(meta[['latitude', 'longitude']].values)

path = os.path.join(box_dir, 'west_coast_buoy_coords.npy')
coords = np.load(path)

_, pos = tree.query(coords[:, ::-1])

buoy_meta = meta.loc[pos]
buoy_meta['latitude'] = coords[:, 1]
buoy_meta['longitude'] = coords[:, 0]

In [17]:
meta_arr = to_records_array(buoy_meta)

out_path = os.path.join(box_dir, 'west_coast_buoy_meta.npy')
np.save(out_path, meta_arr)
meta_arr

rec.array([( 257.4864, 48.494   , -124.728 ,  10718.147  , -8, b'Washington'),
           (  40.6478, 46.858   , -124.244 ,   8284.94   , -8, b'Federal'),
           (  63.2   , 46.2     , -124.2   ,  10457.422  , -8, b'Federal'),
           (  52.4622, 41.852   , -124.382 ,  12598.53   , -8, b'California'),
           ( 114.2939, 40.888   , -124.357 ,  15849.143  , -8, b'Federal'),
           (  41.7298, 40.753   , -124.313 ,   5679.7046 , -8, b'Federal'),
           ( 405.0224, 40.72    , -124.531 ,  20156.863  , -8, b'Federal'),
           ( 329.6658, 39.235   , -123.974 ,  15693.672  , -8, b'Federal'),
           (  12.6779, 37.786   , -122.634 ,   9242.176  , -8, b'Federal'),
           (  53.4392, 37.755   , -122.839 ,  15359.204  , -8, b'Federal'),
           ( 153.1093, 36.760002, -121.949 ,  12769.475  , -8, b'California'),
           (  20.    , 36.626   , -121.907 ,    482.70996, -8, b'California'),
           ( 372.0182, 36.341   , -122.102 ,  17693.432  , -8, b'Federal'),


In [18]:
meta_path = os.path.join(box_dir, 'west_coast_buoy_meta.npy')
meta_data = np.load(meta_path)

m_size = sys.getsizeof(meta_data) * 10**-6
print('meta size = {:.3f} MB'.format(m_size))

m_chunks = get_chunk_size(meta_data, thresh=0.1)

meta size = 0.003 MB
Chunk size (92, ) = 0.00 MB


In [19]:
meta_path = os.path.join(box_dir, 'west_coast_buoy_meta.npy')
meta_data = np.load(meta_path, allow_pickle=True)
lat_lon = pd.DataFrame(meta_data)
lat_lon = lat_lon[['latitude', 'longitude']].values.copy()

c_size = sys.getsizeof(lat_lon) * 10**-6
print('lat_lon size = {:.3f} MB'.format(c_size))

c_chunks = get_chunk_size(lat_lon, thresh=0.1)

lat_lon size = 0.001 MB
Chunk size (92, ) = 0.00 MB


In [30]:
dset_arr = np.ones((8760, 57), dtype='float32')
dset_size = sys.getsizeof(dset_arr) * 10**-6
print('dset size = {:.3f} MB'.format(dset_size))

dset size = 1.997 MB


In [56]:
dset_arr = np.ones((8760, 29, 36, 57), dtype='float32')
dset_size = sys.getsizeof(dset_arr) * 10**-6
print('energy size = {:.3f} MB'.format(dset_size))

dset_arr = np.ones((12*7*24, 6, 8, 5), dtype='float32')
dset_size = sys.getsizeof(dset_arr) * 10**-6
print('energy size = {:.3f} MB'.format(dset_size))

energy size = 2085.160 MB
energy size = 1.936 MB


In [25]:
names = {'depth': 'water_depth'}

descriptions = {'depth': 'Grid node depth',
         'mean_wave_direction': 'Direction Normal to the Wave Crests',
         'significant_wave_height': 'Calculated as the zeroth spectral moment (i.e., H_m0)',
         'mean_absolute_period': 'Resolved Spectral Moment (m_0/m_1)',
         'peak_period': 'The period associated with the maximum value of the wave energy spectrum',
         'mean_zero-crossing_period': 'Total wave energy flux from all directions',
         'energy_period': 'Spectral width characterizes the relative spreading of energy in the wave spectrum. Large values indicate a wider spectral peak',
         'directionality_coefficient': 'Fraction of total wave energy travelling in the "direction of maximum wave power" direction',
         'maximum_energy_direction': 'The direction from which the most wave energy is travelling',
         'omni-directional_wave_power': 'Total wave energy flux from all directions',
         'spectral_width': 'Spectral width characterizes the relative spreading of energy in the wave spectrum. Large values indicate a wider spectral peak',
         'directional_wave_spectrum': 'Variance density over the i^th frequency and j^th direction (m^2Hz^-1deg^-1)',
         'direction': 'direction of wave propagation based on epsg:4326 (deg)',
         'frequency': 'i^th discrete frequency (Hz)',
         'frequency_bin_edges': 'Frequency bin definition: [Low Edge, High Edge)'}

units = {'depth': 'm',
         'mean_wave_direction': 'deg',
         'significant_wave_height': 'm',
         'mean_absolute_period': 's',
         'peak_period': 's',
         'mean_zero-crossing_period': 's',
         'energy_period': 's',
         'directionality_coefficient': '',
         'maximum_energy_direction': 'deg',
         'omni-directional_wave_power': 'W/m',
         'spectral_width': '',
         'directional_wave_spectrum': 'm^2 Hz^-1 deg^-1',
         'direction': 'deg',
         'frequency': 'Hz',
         'frequency_bin_edges': 'Hz'}

src_names = {k : k for k in names}
src_names['directional_wave_spectrum'] = 'energy'

SWAN_names = {'depth': 'DEPTH',
         'mean_wave_direction': 'DIR',
         'significant_wave_height': 'HSIGN',
         'mean_absolute_period': 'PER',
         'peak_period': 'RTP',
         'mean_zero-crossing_period': 'TMM10',
         'energy_period': 'TM02',
         'directionality_coefficient': '',
         'maximum_energy_direction': '',
         'omni-directional_wave_power': '',
         'spectral_width': '',
         'directional_wave_spectrum': 'energy',
         'frequency': 'frequency',
         'direction': 'direction',
         'frequency_bin_edges': ''}

IEC_names = {'depth': 'h',
         'mean_wave_direction': 'Sigma',
         'significant_wave_height': 'H_s',
         'mean_absolute_period': 'T_p',
         'peak_period': 'T_p',
         'mean_zero-crossing_period': 'T_02',
         'energy_period': 'T_e',
         'directionality_coefficient': 'd',
         'maximum_energy_direction': 'Jsigma_Jmax',
         'omni-directional_wave_power': 'J',
         'spectral_width': 'epsilon_o',
         'directional_wave_spectrum': 'S_ij',
         'direction': 'Sigma',
         'frequency': 'f_i',
         'frequency_bin_edges': ''}

references = {'depth': 'SWAN Manual',
         'mean_wave_direction': 'SWAN Manual',
         'significant_wave_height': 'SWAN Manual, IEC62600-101',
         'mean_absolute_period': 'SWAN Manual',
         'peak_period': 'SWAN Manual',
         'mean_zero-crossing_period': 'SWAN Manual, IEC62600-101',
         'energy_period': 'IEC62600-101',
         'directionality_coefficient': 'IEC62600-101',
         'maximum_energy_direction': 'IEC62600-101',
         'omni-directional_wave_power': 'IEC62600-101',
         'spectral_width': 'IEC62600-101',
         'directional_wave_spectrum': 'IEC62600-101',
         'direction': 'IEC62600-101',
         'frequency': 'IEC62600-101',
         'frequency_bin_edges': 'IEC62600-101'}

dimensions = {'depth': ['position'],
         'mean_wave_direction': ['time', 'position'],
         'significant_wave_height': ['time', 'position'],
         'mean_absolute_period': ['time', 'position'],
         'peak_period': ['time', 'position'],
         'mean_zero-crossing_period': ['time', 'position'],
         'energy_period': ['time', 'position'],
         'directionality_coefficient': ['time', 'position'],
         'maximum_energy_direction': ['time', 'position'],
         'omni-directional_wave_power': ['time', 'position'],
         'spectral_width': ['time', 'position'], 
         'directional_wave_spectrum': ['time', 'frequency', 'direction', 'position'],
         'direction': ['direction'],
         'frequency': ['frequency'],
         'frequency_bin_edges': ['frequency', '[Low Edge, High Edge)']}

In [24]:
path = os.path.join(cwd, 'h5_attrs/west_coast_buoy_h5_attrs.json')
buoy_attrs = pd.read_json(path)
buoy_attrs

,attrs,dtype,chunks
coordinates,NaN,float32,None
depth,NaN,float32,None
direction,NaN,float32,None
directional_wave_spectrum,NaN,float32,None
energy_period,NaN,float32,"[549, 12]"
frequency,NaN,float32,None
maximum_energy_direction,NaN,float32,"[549, 12]"
mean_wave_direction,NaN,float32,"[549, 12]"
mean_zero-crossing_period,NaN,float32,"[549, 12]"
omni-directional_wave_power,NaN,float64,"[549, 6]"


In [29]:
path = os.path.join(cwd, 'h5_attrs/west_coast_buoy_h5_attrs.json')
buoy_attrs = pd.read_json(path)

buoy_attrs['name'] = None
buoy_attrs['attrs'] = None

for var, row in buoy_attrs.iterrows():

    if var == 'time_index':
        buoy_attrs.at[var, 'attrs'] = {'freq': '1h', 'timezone': 'UTC', 'units': 'GMT', 'dimensions': ['time']}
        buoy_attrs.at[var, 'chunks'] = None
    elif var == 'coordinates':
        buoy_attrs.at[var, 'chunks'] = None
        attrs  = {'description': '(latitude, longitude) using Datum: NAD83',
                  'src_name': '(Xp, Yp)',
                  'units': '(deg, deg)',
                  'dimensions': ['position']}
        buoy_attrs.at[var, 'attrs'] = attrs
    else:
        if var == 'directional_wave_spectrum':
            chunks = (8*7*24, 5, 4, 20)
        else:
            chunks = None

        attrs = {'description': descriptions[var],
                 'dimensions': dimensions[var],
                 'units': units[var],
                 'SWAN_name': SWAN_names[var],
                 'IEC_name': IEC_names[var]}
        src_name = src_names.get(var)
        if src_name:
            attrs['src_name'] = src_name

        buoy_attrs.at[var, 'chunks'] = chunks
        buoy_attrs.at[var, 'dtype'] = 'float32'
        buoy_attrs.at[var , 'name'] = names.get(var)
        buoy_attrs.at[var, 'attrs'] = attrs

# Meta data
buoy_attrs.at['meta', 'chunks'] = None
buoy_attrs.at['meta', 'dtype'] = None
buoy_attrs.at['meta', 'name'] = None
buoy_attrs.at['meta', 'attrs'] = {'dimensions': ['position']}

# Global attributes
buoy_attrs.at['global', 'chunks'] = None
buoy_attrs.at['global', 'dtype'] = None
buoy_attrs.at['global', 'name'] = None
buoy_attrs.at['global', 'attrs'] = {
    'ref_SWAN-Manual': "SWAN Team, SWAN: User Manual, Delft University of Technology, Delft, The Netherlands, Cycle III Version 41.31, 2019.",
    "ref_IEC62600-101": "International Electrotechnical Commission, Marine energy - Wave, tidal and other water current converters - Part 101: Wave energy resource assessment and characterization, Technical Specification 62600–101, 2015.",
    "ref_Wu-Wang-Yang-Garcia-Medina-2020": "W.C. Wu, T. Wang, Z. Yang, and G. García-Medina, “Development and validation of a high-resolution regional wave hindcast model for U.S. West Coast wave resource characterization,” Renewable Energy, vol. 152, pp. 736–753, Jun. 2020.",
    "source": "PNNL2019", "version": "v1.0.0"}

path = os.path.join(cwd, 'hsds_attrs/west_coast_buoy_hsds_attrs.json')
buoy_attrs.to_json(path, indent=4)
buoy_attrs

,attrs,dtype,chunks,name
coordinates,"{'description': '(latitude, longitude) using D...",float32,None,None
depth,"{'description': 'Grid node depth', 'dimensions...",float32,None,water_depth
direction,{'description': 'direction of wave propagation...,float32,None,None
directional_wave_spectrum,{'description': 'Variance density over the i^t...,float32,"(1344, 5, 4, 20)",None
energy_period,{'description': 'Spectral width characterizes ...,float32,None,None
frequency,{'description': 'i^th discrete frequency (Hz)'...,float32,None,None
maximum_energy_direction,{'description': 'The direction from which the ...,float32,None,None
mean_wave_direction,{'description': 'Direction Normal to the Wave ...,float32,None,None
mean_zero-crossing_period,{'description': 'Total wave energy flux from a...,float32,None,None
omni-directional_wave_power,{'description': 'Total wave energy flux from a...,float32,None,None
